# Foundations of Computer Science Project
## Academic Year 2022-23
### Eleonora Brambatti 858098, Camilla Fracchia 898235, Marta Privitera 898017

You have to work on the Dogs adoptions dataset.

It contains three files:

- dogs.csv, *shortly dogs*
- dogTravel.csv, *shortly travels*
- NST-EST2021-POP.csv

### Notes

- It is mandatory to use GitHub for developing the project.
- The project must be a jupyter notebook.
- There is no restriction on the libraries that can be used, nor on the Python version.
- All questions on the project must be asked in a public channel on Zulip.
- At most 3 students can be in each group. You must create the groups by yourself.
- You do not have to send me the project before the discussion.

We import the pandas library in order to manage .csv files

In [614]:
import pandas as pd

We access and save the dogs dataset as "dogs"

In [615]:
dogs=pd.read_csv('dogs.csv', sep=',')
dogs

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,Lander,WY,82520,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34
58176,44457061,WY24,https://www.petfinder.com/dog/harley-44457061/...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,65,245.90
58177,42865848,WY20,https://www.petfinder.com/dog/echo-42865848/wy...,Dog,Dog,Border Collie,NaN,False,False,NaN,...,Glenrock,WY,82637,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,184.06
58178,42734734,WY24,https://www.petfinder.com/dog/simon-42734734/w...,Dog,Dog,Boxer,Mixed Breed,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,58,61.05


We check all variables types

In [616]:
dogs.dtypes

id                   int64
org_id              object
url                 object
type.x              object
species             object
breed_primary       object
breed_secondary     object
breed_mixed           bool
breed_unknown         bool
color_primary       object
color_secondary     object
color_tertiary      object
age                 object
sex                 object
size                object
coat                object
fixed                 bool
house_trained         bool
declawed           float64
special_needs         bool
shots_current         bool
env_children        object
env_dogs            object
env_cats            object
name                object
status              object
posted              object
contact_city        object
contact_state       object
contact_zip         object
contact_country     object
stateQ              object
accessed            object
type.y              object
description         object
stay_duration        int64
stay_cost          float64
d

We noticed that the variables 'accessed' and 'posted' are in a format too generic: in fact, by doing the group by we observe that there are names of cities, incompatible with the time format

In [617]:
dogs.groupby('posted', as_index=False).size() 

,posted,size
0,2003-05-18T00:00:00+0000,1
1,2004-02-23T00:00:00+0000,1
2,2004-02-29T00:00:00+0000,1
3,2005-05-30T00:00:00+0000,1
4,2005-09-26T00:00:00+0000,1
...,...,...
48368,Rustburg,1
48369,Saugerties,2
48370,Seattle,7
48371,Silver Spring,1


In [618]:
import re

In [619]:
listposted = list(dogs['posted'])
listposted

['2019-09-20T16:37:59+0000',
 '2019-09-20T16:24:57+0000',
 '2019-09-20T14:10:11+0000',
 '2019-09-20T10:08:22+0000',
 '2019-09-20T06:48:30+0000',
 '2019-09-20T06:43:59+0000',
 '2019-09-20T06:42:30+0000',
 '2019-09-20T06:40:08+0000',
 '2019-09-20T06:37:05+0000',
 '2019-09-20T05:00:51+0000',
 '2019-09-20T04:59:44+0000',
 '2019-09-20T04:56:12+0000',
 '2019-09-20T04:54:11+0000',
 '2019-09-20T03:04:00+0000',
 '2019-09-20T02:18:49+0000',
 '2019-09-20T02:18:48+0000',
 '2019-09-20T02:18:48+0000',
 '2019-09-20T02:18:48+0000',
 '2019-09-20T02:18:48+0000',
 '2019-09-20T02:18:47+0000',
 '2019-09-20T02:18:47+0000',
 '2019-09-20T02:18:47+0000',
 '2019-09-20T02:18:47+0000',
 '2019-09-20T02:14:12+0000',
 '2019-09-20T02:05:53+0000',
 '2019-09-20T01:59:01+0000',
 '2019-09-20T01:58:07+0000',
 '2019-09-20T01:48:49+0000',
 '2019-09-20T01:37:08+0000',
 '2019-09-20T01:29:02+0000',
 '2019-09-20T01:18:34+0000',
 '2019-09-20T01:14:09+0000',
 '2019-09-19T23:18:14+0000',
 '2019-09-19T21:17:30+0000',
 '2019-09-19T2

In [620]:
wrong_records = []

for line in listposted:
    occ = re.search('^[\d+-]+\d+T[\d+:]+\d+\+\d+$', line)
    if not occ:
        wrong_records.append(line)
len(wrong_records)

33

We check that there are 33 wrong rows

In [621]:
dogs.groupby('accessed', as_index=False).size() 

,accessed,size
0,2019-09-20,58147


**Revision of the dataset**

We extract the 33 rows with the missing values of 'accessed' column

In [622]:
dogs_1=dogs[dogs['accessed'].isnull()]
dogs_1

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,NV,89146,US,89009,2019-09-20,NaN,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,AZ,85249,US,AZ,2019-09-20,NaN,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,NY,12220,US,CT,2019-09-20,NaN,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,NY,12220,US,CT,2019-09-20,NaN,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,NY,12220,US,CT,2019-09-20,NaN,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
12600,45229004,NY1436,https://www.petfinder.com/dog/elmo-momo-452290...,Dog,Dog,American Bulldog,NaN,True,False,NaN,...,NY,12477,US,CT,2019-09-20,NaN,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
12613,45227052,NY1436,https://www.petfinder.com/dog/bianca-pinky-452...,Dog,Dog,Mixed Breed,NaN,False,False,White / Cream,...,NY,12477,US,CT,2019-09-20,NaN,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
17619,45569380,CA1209,https://www.petfinder.com/dog/baby-girl-455693...,Dog,Dog,Maltese,NaN,False,False,White / Cream,...,VA,20136,US,DC,2019-09-20,NaN,Dog,This 10-year young senior is very sweet and lo...,76,263.63
18611,44694387,MD295,https://www.petfinder.com/dog/king-bert-bertie...,Dog,Dog,Fox Terrier,Chihuahua,True,False,Bicolor,...,MD,20905,US,DC,2019-09-20,NaN,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
19747,36978896,VA127,https://www.petfinder.com/dog/maddie-cutie-pat...,Dog,Dog,Alaskan Malamute,NaN,False,False,Bicolor,...,PA,17325,US,DC,2019-09-20,NaN,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


We can see that from 'id' to 'color_primary' variables every record is correct

In [623]:
dogs_1.loc[:,'color_secondary':'shots_current']

,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current
644,NaN,NaN,Young,Male,Large,NaN,False,False,NaN,False,False
5549,White / Cream,NaN,Adult,Female,Large,Short,True,True,NaN,False,True
10888,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True
11983,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True
12495,White / Cream,NaN,Senior,Female,Medium,Short,True,True,NaN,False,True
12600,NaN,NaN,Senior,Male,Large,Short,True,True,NaN,False,True
12613,NaN,NaN,Senior,Female,Medium,Short,True,True,NaN,False,True
17619,NaN,NaN,Senior,Female,Small,Short,True,True,NaN,False,True
18611,NaN,NaN,Young,Male,Small,Short,True,False,NaN,False,True
19747,NaN,NaN,Adult,Female,Large,NaN,True,True,NaN,False,True


We can see that from 'color_secondary' to 'shots_current' variables every record is correct

In [624]:
dogs_1.loc[:,'env_children':'accessed']

,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed
644,NaN,NaN,NaN,"Gunther :Gunny\"",Meet handsome 3 year old Gunt...",2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20,NaN
5549,NaN,NaN,False,"ANNABELLE \ANNIE\"",""You can fill out an adopti...",2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20,NaN
10888,True,True,True,"PEPPER \Courtesy listing\"",""This is Pepper. He...",2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
11983,NaN,NaN,False,"COOPER \courtesy listing\"",""Cooper is 13 years...",2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
12495,False,False,NaN,"DAISY \courtesy listing\"",""•Basset Hound, fema...",2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
12600,True,True,NaN,"Elmo \MoMo\"",""Hello i&#039;m MoMo or Elmo , 7 ...",2019-07-11T20:34:42+0000,Saugerties,NY,12477,US,CT,2019-09-20,NaN
12613,True,True,NaN,"Bianca \Pinky\"",""Hello I&#039;m Bianca, a fema...",2019-07-11T14:16:38+0000,Saugerties,NY,12477,US,CT,2019-09-20,NaN
17619,True,True,NaN,"\Baby Girl\"",""This 10-year young senior is ver...",2019-08-10T16:00:35+0000,Bristow,VA,20136,US,DC,2019-09-20,NaN
18611,False,NaN,False,"King Bert \Bertie\"",""&quot;Bertie&quot; came t...",2019-05-14T21:09:27+0000,Silver Spring,MD,20905,US,DC,2019-09-20,NaN
19747,NaN,NaN,False,"Maddie \Cutie Patootie!\"",""Maddie is our littl...",2016-12-15T13:33:43+0000,Gettysburg,PA,17325,US,DC,2019-09-20,NaN


We can see that from the 'status' to the 'accessed' variables the records are shifted by one column

In [625]:
dogs_1.loc[:,'type.y':'stay_cost']

,type.y,description,stay_duration,stay_cost
644,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,Dog,You can fill out an adoption application onlin...,80,130.77
10888,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
12600,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
12613,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
17619,Dog,This 10-year young senior is very sweet and lo...,76,263.63
18611,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
19747,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


We can see that from 'type.y' to 'stay_cost' variables every record is correct

We create a copy of the dogs_1 dataset that we are going to manipulate. On this new dataset (dogs_tmp) we shift the wrong records

In [626]:
dogs_tmp=dogs_1.copy()
dogs_tmp.loc[:,('accessed')]=dogs_1.loc[:,('stateQ')]
dogs_tmp.loc[:,('stateQ')]=dogs_1.loc[:,('contact_country')]
dogs_tmp.loc[:,('contact_country')]=dogs_1.loc[:,('contact_zip')]
dogs_tmp.loc[:,('contact_zip')]=dogs_1.loc[:,('contact_state')]
dogs_tmp.loc[:,('contact_state')]=dogs_1.loc[:,('contact_city')]
dogs_tmp.loc[:,('contact_city')]=dogs_1.loc[:,('posted')]
dogs_tmp.loc[:,('posted')]=dogs_1.loc[:,('status')]
dogs_tmp.loc[:,('status')]=''

#dogs_1.head()
dogs_tmp.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61


We drop the rows with the wrong records from the dogs dataset. We obtain the dogs_2 dataset

In [627]:
list_indexes=dogs[dogs['accessed'].isnull()].index
list_indexes=list(list_indexes)

dogs_2=dogs.drop(labels=list_indexes, axis=0)
dogs_2

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,Lander,WY,82520,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34
58176,44457061,WY24,https://www.petfinder.com/dog/harley-44457061/...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,65,245.90
58177,42865848,WY20,https://www.petfinder.com/dog/echo-42865848/wy...,Dog,Dog,Border Collie,NaN,False,False,NaN,...,Glenrock,WY,82637,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,184.06
58178,42734734,WY24,https://www.petfinder.com/dog/simon-42734734/w...,Dog,Dog,Boxer,Mixed Breed,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,58,61.05


We concatenate the dogs_2 and dogs_tmp datasets and we create the dogs_ok dataset. We control that the dogs_ok dataset has the same shape of the dogs dataset

In [628]:
dogs_ok=pd.concat([dogs_2, dogs_tmp])
dogs_ok = dogs_ok.sort_index()
dogs_ok

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,Lander,WY,82520,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34
58176,44457061,WY24,https://www.petfinder.com/dog/harley-44457061/...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,65,245.90
58177,42865848,WY20,https://www.petfinder.com/dog/echo-42865848/wy...,Dog,Dog,Border Collie,NaN,False,False,NaN,...,Glenrock,WY,82637,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,184.06
58178,42734734,WY24,https://www.petfinder.com/dog/simon-42734734/w...,Dog,Dog,Boxer,Mixed Breed,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,58,61.05


In [629]:
dogs_ok.shape

(58180, 37)

We control that now the 'accessed' column has no missing values

In [630]:
dogs_ok.groupby('accessed', as_index=False).size()

,accessed,size
0,2019-09-20,58180


Now we can change the 'accessed' and 'posted' columns types

In [631]:
dogs_ok['accessed']=pd.to_datetime(dogs_ok['accessed'], format='%Y-%m-%d')

In [632]:
dogs_ok['posted']=pd.to_datetime(dogs_ok['posted'], format='%Y-%m-%dT%H:%M:%S%z')

In [633]:
dogs_ok.dtypes

id                               int64
org_id                          object
url                             object
type.x                          object
species                         object
breed_primary                   object
breed_secondary                 object
breed_mixed                       bool
breed_unknown                     bool
color_primary                   object
color_secondary                 object
color_tertiary                  object
age                             object
sex                             object
size                            object
coat                            object
fixed                             bool
house_trained                     bool
declawed                       float64
special_needs                     bool
shots_current                     bool
env_children                    object
env_dogs                        object
env_cats                        object
name                            object
status                   

**1. Extract all dogs with status that is not *adoptable***

In [634]:
dogs_ok[dogs_ok['status'] !='adoptable']

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
12600,45229004,NY1436,https://www.petfinder.com/dog/elmo-momo-452290...,Dog,Dog,American Bulldog,NaN,True,False,NaN,...,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
12613,45227052,NY1436,https://www.petfinder.com/dog/bianca-pinky-452...,Dog,Dog,Mixed Breed,NaN,False,False,White / Cream,...,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
17619,45569380,CA1209,https://www.petfinder.com/dog/baby-girl-455693...,Dog,Dog,Maltese,NaN,False,False,White / Cream,...,Bristow,VA,20136,US,DC,2019-09-20,Dog,This 10-year young senior is very sweet and lo...,76,263.63
18611,44694387,MD295,https://www.petfinder.com/dog/king-bert-bertie...,Dog,Dog,Fox Terrier,Chihuahua,True,False,Bicolor,...,Silver Spring,MD,20905,US,DC,2019-09-20,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
19747,36978896,VA127,https://www.petfinder.com/dog/maddie-cutie-pat...,Dog,Dog,Alaskan Malamute,NaN,False,False,Bicolor,...,Gettysburg,PA,17325,US,DC,2019-09-20,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


**2. For each (primary) breed, determine the number of dogs**

In [635]:
dogs_ok.groupby('breed_primary', as_index=False).size()

,breed_primary,size
0,Affenpinscher,17
1,Afghan Hound,4
2,Airedale Terrier,19
3,Akbash,3
4,Akita,181
...,...,...
211,Wirehaired Pointing Griffon,1
212,Wirehaired Terrier,60
213,Xoloitzcuintli / Mexican Hairless,11
214,Yellow Labrador Retriever,158


**3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.**

We create a copy of the dataset (dogs_3) for this point

In [636]:
dogs_3=dogs_ok.copy()

We create the new columns 'mixed' and 'not_mixed' in the dogs_3 dataset

In [637]:
dogs_3['mixed']=dogs_3['breed_secondary']=='Mixed Breed'
dogs_3['not_mixed']=dogs_3['breed_secondary']!='Mixed Breed'
dogs_3.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,mixed,not_mixed
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81,True,False
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07,True,False
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51,False,True
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,89048,US,89009,2019-09-20,Dog,NaN,62,145.83,False,True
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09,False,True


We create a new dataset with the columns we are interested in and then we obtain the ratio

In [638]:
dogs_mixed_not_mixed=dogs_3[['breed_primary','mixed', 'not_mixed']].groupby('breed_primary', as_index=False).sum()
dogs_mixed_not_mixed['ratio']=dogs_mixed_not_mixed['mixed']/dogs_mixed_not_mixed['not_mixed']
dogs_mixed_not_mixed.head()

,breed_primary,mixed,not_mixed,ratio
0,Affenpinscher,1,16,0.062500
1,Afghan Hound,0,4,0.000000
2,Airedale Terrier,1,18,0.055556
3,Akbash,0,3,0.000000
4,Akita,6,175,0.034286


**4. For each (primary) breed, determine the earliest and the latest posted timestamp.**

We create two new datasets earliest and latest which contain earliest and latest posted timestamp.
Then we merge them on breed_primary column in a new dataset (earliest_latest).

In [639]:
earliest=dogs_ok[['breed_primary','posted']].groupby('breed_primary').min()
latest=dogs_ok[['breed_primary','posted']].groupby('breed_primary').max()

earliest_latest=pd.merge(earliest, latest, on='breed_primary', suffixes=('_earliest', '_latest'))
earliest_latest.rename(columns = {'posted_earliest':'earliest', 'posted_latest':'latest'}, inplace = True)
earliest_latest

,earliest,latest
breed_primary,,
Affenpinscher,2012-03-08 10:27:33+00:00,2019-09-14 10:10:51+00:00
Afghan Hound,2017-06-29 23:28:51+00:00,2019-07-27 00:38:48+00:00
Airedale Terrier,2014-06-13 12:59:36+00:00,2019-09-19 18:40:39+00:00
Akbash,2019-07-21 00:35:59+00:00,2019-08-23 17:11:04+00:00
Akita,2012-03-03 09:31:08+00:00,2019-09-20 15:19:57+00:00
...,...,...
Wirehaired Pointing Griffon,2016-06-29 20:03:55+00:00,2016-06-29 20:03:55+00:00
Wirehaired Terrier,2012-11-27 14:07:54+00:00,2019-09-19 22:52:45+00:00
Xoloitzcuintli / Mexican Hairless,2007-02-01 00:00:00+00:00,2019-09-08 11:15:54+00:00


**5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?**

In [640]:
dogs_ok.columns

Index(['id', 'org_id', 'url', 'type.x', 'species', 'breed_primary',
       'breed_secondary', 'breed_mixed', 'breed_unknown', 'color_primary',
       'color_secondary', 'color_tertiary', 'age', 'sex', 'size', 'coat',
       'fixed', 'house_trained', 'declawed', 'special_needs', 'shots_current',
       'env_children', 'env_dogs', 'env_cats', 'name', 'status', 'posted',
       'contact_city', 'contact_state', 'contact_zip', 'contact_country',
       'stateQ', 'accessed', 'type.y', 'description', 'stay_duration',
       'stay_cost'],
      dtype='object')

We create a copy of the dataset (dogs_5) for this point. We create the new columns 'male' and 'female' in the dogs_5 dataset

In [641]:
dogs_5=dogs_ok.copy()

dogs_5['male'] = dogs_5['sex']=='Male' 

dogs_5['female']= dogs_5['sex']=='Female'

We create a new dataset (male_female) with the sum of male and female in each state and then we calculate the sex_imbalance

In [642]:
male_female=dogs_5[['contact_state','male', 'female']].groupby('contact_state').sum()

male_female['sex_imbalance']=male_female['male']-male_female['female']
male_female

,male,female,sex_imbalance
contact_state,,,
AK,8,7,1
AL,712,716,-4
AR,344,351,-7
AZ,1181,1067,114
CA,887,777,110
CO,861,912,-51
CT,740,682,58
DC,160,176,-16
DE,148,148,0


We find the State with the max sex imbalance

In [643]:
male_female[male_female['sex_imbalance']==male_female['sex_imbalance'].max()]

,male,female,sex_imbalance
contact_state,,,
OH,1439,1234,205


**6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.**

In [644]:
dogs_ok[['age','size','stay_duration', 'stay_cost']].groupby(['age','size']).mean()

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

**7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.**

We read the Travel dataset and we set the index

In [645]:
Travel= pd.read_csv('dogTravel.csv')# read the dog travel file

Travel.set_index('index', inplace=True) #we find the 'index' is queal to the implicit index, we set it as the index of the entire dataframe
Travel.head()


,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


We create a dataset with the number of cities for each dog

In [646]:
num_travel=Travel[['id','contact_city']].groupby('id').count()  
num_travel.head()

,contact_city
id,
8619716,2
9317153,1
12134429,1
14355301,1
16313278,1


We merge the dataset with the breed columns and the new dataset num_travel and we obtain the traveller_dogs dataset

In [647]:
traveller_dogs=pd.merge(dogs_ok[['id','breed_primary','breed_secondary','breed_mixed']],num_travel, on='id')
traveller_dogs.rename(columns={"contact_city": "number_of_travels"}, inplace=True)
traveller_dogs[traveller_dogs['number_of_travels']>=3] 

,id,breed_primary,breed_secondary,breed_mixed,number_of_travels
68,45642530,Jindo,NaN,False,4
236,46039420,Border Collie,Retriever,True,4
318,40036107,Pit Bull Terrier,NaN,True,3
578,45851842,Labrador Retriever,NaN,False,3
596,45841145,Mixed Breed,NaN,True,3
...,...,...,...,...,...
4004,41144335,Chihuahua,NaN,False,3
4011,40103682,Rat Terrier,Jack Russell Terrier,True,3
4014,38664932,Pit Bull Terrier,NaN,False,3
4015,38495992,Pit Bull Terrier,NaN,False,3


**8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.**

In [648]:
import numpy as np
Travel.head()

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


We create two list with the records of 'found' and 'manual' columns

In [649]:
manual=list(Travel['manual']) 
found=list(Travel['found']) 

We create a new list with the records of found where manual is missing otherwise with manual values

In [650]:
found1=[]
for i in range(len(manual)):  
    if str(manual[i]) == 'nan':     
        found1.append(found[i])      

    else:    
        found1.append(manual[i]) 
found1

['Arkansas',
 'Bahamas',
 'Maryland',
 'Adaptil',
 'Afghanistan',
 'Afghanistan',
 'Ohio',
 'Ohio',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',


We replace the records of 'found' column with the values of found1 list

In [651]:
Travel['found'] = found1 
Travel

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN


**9. For each state, compute the ratio between the number of travels and the population.**

We check that contact_state values are correct in Travel dataset

In [652]:
Travel.groupby('contact_state').size() #controlliamo che tutte le sigle tornino rispetto al dizionario sottostante
#17325 è PA
#unico che non torna è NB poichè è uno stato Canadese

contact_state
17325      10
AL         75
AR         10
AZ         70
CA         28
CO        103
CT         90
DC        112
DE         57
FL        133
GA        109
IA         22
IL         37
IN         65
KS          3
KY         31
LA          5
MA        107
MD        379
ME        361
MI         67
MN        190
MO         42
MS          6
NB          2
NC        187
NH         44
NJ        552
NM         82
NV         20
NY        490
OH        177
OK         31
OR         32
PA        316
RI        130
SC         21
TN        112
TX          2
UT         66
VA       1025
VT         49
WA        634
WI         83
WV         27
dtype: int64

We read the state_population dataset

In [653]:

state_population= pd.read_csv('NST-EST2021-POP.csv',names=['state','population'])
state_population.head()

,state,population
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223


We change the State code with the corresponding State name.

In [654]:
us_state_abbrev = {
            'AL': 'Alabama',
            'AK': 'Alaska',
            'AZ': 'Arizona',
            'AR': 'Arkansas',
            'CA': 'California',
            'CO': 'Colorado',
            'CT': 'Connecticut',
            'DE': 'Delaware',
            'FL': 'Florida',
            'GA': 'Georgia',
            'HI': 'Hawaii',
            'ID': 'Idaho',
            'IL': 'Illinois',
            'IN': 'Indiana',
            'IA': 'Iowa',
            'KS': 'Kansas',
            'KY': 'Kentucky',
            'LA': 'Louisiana',
            'ME': 'Maine',
            'MD': 'Maryland',
            'MA': 'Massachusetts',
            'MI': 'Michigan',
            'MN': 'Minnesota',
            'MS': 'Mississippi',
            'MO': 'Missouri',
            'MT': 'Montana',
            'NB': 'New Brunswick',
            'NE': 'Nebraska',
            'NV': 'Nevada',
            'NH': 'New Hampshire',
            'NJ': 'New Jersey',
            'NM': 'New Mexico',
            'NY': 'New York',
            'NC': 'North Carolina',
            'ND': 'North Dakota',
            'OH': 'Ohio',
            'OK': 'Oklahoma',
            'OR': 'Oregon',
            'PA': 'Pennsylvania',
            'RI': 'Rhode Island',
            'SC': 'South Carolina',
            'SD': 'South Dakota',
            'TN': 'Tennessee',
            'TX': 'Texas',
            'UT': 'Utah',
            'VT': 'Vermont',
            'VA': 'Virginia',
            'WA': 'Washington',
            'WV': 'West Virginia',
            'WI': 'Wisconsin',
            'WY': 'Wyoming',
            'DC': 'District of Columbia',
            'MP': 'Northern Mariana Islands',
            'PW': 'Palau',
            'PR': 'Puerto Rico',
            'VI': 'Virgin Islands',
            'AA': 'Armed Forces Americas (Except Canada)',
            'AE': 'Armed Forces Africa/Canada/Europe/Middle East',
            'AP': 'Armed Forces Pacific',
            '17325' : 'Pennsylvania'
        }



Travel['contact_state']=Travel['contact_state'].replace(us_state_abbrev)       
Travel.head()

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,Minnesota,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,Florida,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,45983838,Adamstown,Maryland,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,44475904,Saint Cloud,Minnesota,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,Colorado,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In the new dataset (num_travels) we count the number of travels for each State . Then we merge the num_travels and the state_population datasets in the travel_population datatset

In [655]:
num_travels=Travel[['contact_state','id']].groupby('contact_state').count()

travel_population=pd.merge(num_travels, state_population,left_on='contact_state',right_on='state')
travel_population

,id,state,population
0,75,Alabama,5.024.279
1,70,Arizona,7.151.502
2,10,Arkansas,3.011.524
3,28,California,39.538.223
4,103,Colorado,5.773.714
5,90,Connecticut,3.605.944
6,57,Delaware,989.948
7,112,District of Columbia,689.545
8,133,Florida,21.538.187
9,109,Georgia,10.711.908


We change the values in the population column removing the dots and we change the type and then we calculate the ratio

In [656]:
travel_population['population']=travel_population['population'].str.replace('[^0-9]','')
travel_population['population']=travel_population['population'].astype(float) 

travel_population['ratio']=travel_population['id']/travel_population['population']
travel_population.rename(columns={"id": "num_travels"},inplace=True)
travel_population.set_index('state')

C:\Users\elebr\AppData\Local\Temp\ipykernel_11200\979520101.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  travel_population['population']=travel_population['population'].str.replace('[^0-9]','')


,num_travels,population,ratio
state,,,
Alabama,75,5024279.0,1.492751e-05
Arizona,70,7151502.0,9.788154e-06
Arkansas,10,3011524.0,3.320578e-06
California,28,39538223.0,7.081755e-07
Colorado,103,5773714.0,1.783947e-05
Connecticut,90,3605944.0,2.495879e-05
Delaware,57,989948.0,5.757878e-05
District of Columbia,112,689545.0,1.624259e-04
Florida,133,21538187.0,6.175079e-06


**10.For each dog, compute the number of days from the posted day to the day of last access.**

In [657]:

dogs_10 = dogs_ok.copy()
dogs_10['accessed']=pd.to_datetime(dogs_10['accessed']) 

dogs_10['posted']=dogs_10['posted'].dt.date

dogs_10['posted']=pd.to_datetime(dogs_10['posted']) 

dogs_10['diff']=dogs_10['accessed']-dogs_10['posted']
dogs_10

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,diff
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81,0 days
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07,0 days
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51,0 days
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83,0 days
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,WY,82520,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34,140 days
58176,44457061,WY24,https://www.petfinder.com/dog/harley-44457061/...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,WY,82501,US,WY,2019-09-20,Dog,NaN,65,245.90,160 days
58177,42865848,WY20,https://www.petfinder.com/dog/echo-42865848/wy...,Dog,Dog,Border Collie,NaN,False,False,NaN,...,WY,82637,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,184.06,358 days
58178,42734734,WY24,https://www.petfinder.com/dog/simon-42734734/w...,Dog,Dog,Boxer,Mixed Breed,True,False,NaN,...,WY,82501,US,WY,2019-09-20,Dog,NaN,58,61.05,373 days


**11. Partition the dogs according to the number of weeks from the posted day to the day of last access.**

In [658]:
import math
dogs_10['days'] = dogs_10['accessed']-dogs_10['posted']
dogs_10['days'] = dogs_10['days']/ np.timedelta64(1, 'D')
dogs_10['weeks'] = dogs_10['days']/7
#dogs_10['weeks'] = dogs_10['weeks']
dogs_10['weeks']=dogs_10['weeks'].astype(int)

In [659]:
dogs_10[['id','weeks']].groupby('weeks').count()

,id
weeks,
0,9803
1,6547
2,5764
3,3353
4,2439
...,...
729,1
746,1
811,1


**12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.¶**

In [660]:
dogs_ok

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,Lander,WY,82520,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34
58176,44457061,WY24,https://www.petfinder.com/dog/harley-44457061/...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,65,245.90
58177,42865848,WY20,https://www.petfinder.com/dog/echo-42865848/wy...,Dog,Dog,Border Collie,NaN,False,False,NaN,...,Glenrock,WY,82637,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,184.06
58178,42734734,WY24,https://www.petfinder.com/dog/simon-42734734/w...,Dog,Dog,Boxer,Mixed Breed,True,False,NaN,...,Riverton,WY,82501,US,WY,2019-09-20,Dog,NaN,58,61.05


We select the columns of interest, then we drop the rows with missing descriptions and we sort the values by breed_primary and sex 

In [661]:
dogs_12=dogs_ok[['id', 'breed_primary', 'sex', 'description']]
dogs_12 = dogs_12[dogs_12['description'].notna()]
dogs_12.sort_values(by=['breed_primary', 'sex'], inplace = True)
dogs_12

,id,breed_primary,sex,description
10430,45889013,Affenpinscher,Female,This cutie is very sweet. She is a little shy ...
24994,22427951,Affenpinscher,Female,To see even more click the following link:\n\n...
32156,45970614,Affenpinscher,Female,Elsie.jpgThank you for looking at our availabl...
32235,45871731,Affenpinscher,Female,55351.jpgThank you for looking at our availabl...
51866,45916762,Affenpinscher,Female,Norma Jean is an Affenpinscher mix girl that w...
...,...,...,...,...
55915,45958435,Yorkshire Terrier,Male,For a quicker response please submit a online ...
55992,45922985,Yorkshire Terrier,Male,You can fill out an adoption application onlin...
56450,45466492,Yorkshire Terrier,Male,My name is Juno from California and I am a won...
57517,45985506,Yorkshire Terrier,Male,"2 years, 4 pounds, cute, lovable, friendly, aw..."


We check there are no missing values in the dataset

In [662]:
dogs_12.isnull().sum()

id               0
breed_primary    0
sex              0
description      0
dtype: int64

We create a dataset in which for each breed and sex is calculated the number of dogs

In [676]:
dogs_groups = dogs_12.groupby(['breed_primary', 'sex'], as_index=False).count()[['breed_primary', 'sex', 'id']]
dogs_groups1 = dogs_groups.copy()
dogs_groups1.rename(columns={"id": "num_dogs"},inplace=True)
dogs_groups1

,breed_primary,sex,num_dogs
0,Affenpinscher,Female,7
1,Affenpinscher,Male,7
2,Afghan Hound,Female,1
3,Afghan Hound,Male,3
4,Airedale Terrier,Female,6
...,...,...,...
398,Xoloitzcuintli / Mexican Hairless,Male,7
399,Yellow Labrador Retriever,Female,71
400,Yellow Labrador Retriever,Male,77
401,Yorkshire Terrier,Female,142


We create a two lists with all the breeds and the sex

In [677]:
listbreed = list(dogs_groups1['breed_primary'])
listsex = list(dogs_groups1['sex'])

We create a variable that is the number of different groups

In [679]:
groups = len(listbreed)
groups

403

We create a list with the id of the dogs from the dogs_12 dataset

In [680]:
listid = list(dogs_12['id'])

We create a list with the numbers that are the extreme values of the intervals of dogs groups

In [681]:
rangelist = list(dogs_groups1['num_dogs'])

rangelist_1 = []
ran = 0
for i in rangelist:
    ran = ran + i
    rangelist_1.append(ran)

rangelist_2 = rangelist_1.copy()
rangelist_2.insert(0, 0)
rangelist_2

[0,
 7,
 14,
 15,
 18,
 24,
 34,
 36,
 37,
 114,
 200,
 224,
 264,
 650,
 1191,
 1210,
 1231,
 1236,
 1246,
 1247,
 1248,
 2103,
 2945,
 2992,
 3049,
 3453,
 3871,
 3889,
 3903,
 4183,
 4536,
 4540,
 4547,
 4574,
 4597,
 4672,
 4809,
 5434,
 6182,
 6183,
 6186,
 6189,
 6190,
 6192,
 6193,
 6238,
 6278,
 6279,
 6281,
 6282,
 6283,
 6291,
 6301,
 6337,
 6377,
 6599,
 6855,
 6945,
 7030,
 7068,
 7106,
 7129,
 7153,
 7161,
 7191,
 7241,
 7243,
 7249,
 7250,
 7581,
 7956,
 7961,
 7982,
 7986,
 7988,
 8061,
 8130,
 8131,
 9001,
 9919,
 9920,
 9921,
 9934,
 9949,
 9961,
 9971,
 10056,
 10122,
 10137,
 10171,
 10211,
 10249,
 10258,
 10265,
 10295,
 10334,
 10341,
 10354,
 10391,
 10407,
 10628,
 10831,
 11043,
 11227,
 11228,
 11230,
 11239,
 11248,
 11256,
 11270,
 12859,
 14629,
 14635,
 14645,
 14646,
 14647,
 14720,
 14791,
 14846,
 14920,
 14922,
 14924,
 14930,
 14941,
 15007,
 15139,
 15238,
 15342,
 15438,
 15600,
 15658,
 15717,
 15718,
 15719,
 15721,
 16083,
 16500,
 16535,
 16570,

We create a list with all the description of the dogs in the dogs_12

In [682]:
descriptionlist = list(dogs_12['description'])
descriptionlist

['This cutie is very sweet. She is a little shy as you can see by her photo. Her owner sadly could no longer care for her. She is confused on why shes with us and would like to find her new forever home.',
 'To see even more click the following link:\n\nThis is a video showing all of the available dogs at the shelter. There is NO Affenpinscher in the shelter at this time.Â\xa0 Adoption hours are 9am-5:30pm on weekdays. Special adoption events occasionally on Saturdays. Saturday hours are 9 am to 1 pm. Adoption Fees are $40. Small dogs and puppies ar $75. This includes spay/neuter, heartworm testing, shot series, internal and external parasite control, rabies shot, license and county required registration. Video Courtesy of Jan Sebald.\n\nPlease note. Adoptions of 36 kennels is full. We have dogs in overflow building 200 for adoptions.',
 'Elsie.jpgThank you for looking at our available pets. \n\nAll cats Felv/FIV tested, if over the age of 6 months will include heart worm test, feline/

We clean the descriptions in the descriptionlist and we add them in the descriptionlist1

In [685]:
import string
descriptionlist1 = []
for line in descriptionlist:
    line = str(line).translate(str.maketrans('', '', string.punctuation))
    line = line.replace('\n'," ")
    descriptionlist1.append(line.lower())
descriptionlist1

['this cutie is very sweet she is a little shy as you can see by her photo her owner sadly could no longer care for her she is confused on why shes with us and would like to find her new forever home',
 'to see even more click the following link  this is a video showing all of the available dogs at the shelter there is no affenpinscher in the shelter at this timeâ\xa0 adoption hours are 9am530pm on weekdays special adoption events occasionally on saturdays saturday hours are 9 am to 1 pm adoption fees are 40 small dogs and puppies ar 75 this includes spayneuter heartworm testing shot series internal and external parasite control rabies shot license and county required registration video courtesy of jan sebald  please note adoptions of 36 kennels is full we have dogs in overflow building 200 for adoptions',
 'elsiejpgthank you for looking at our available pets   all cats felvfiv tested if over the age of 6 months will include heart worm test felinerabies vaccinations dewormed and a mont

We count the number of words for each description in descriptionlist1

In [686]:
from collections import Counter
lenght = []
for i in descriptionlist1:
    c = dict(Counter(i.split()))
    lenght.append(sum(c.values()))
lenght

[43,
 102,
 109,
 109,
 79,
 36,
 7,
 510,
 211,
 144,
 109,
 172,
 175,
 7,
 107,
 75,
 285,
 284,
 128,
 798,
 102,
 225,
 49,
 107,
 149,
 128,
 242,
 376,
 108,
 118,
 69,
 107,
 115,
 224,
 242,
 23,
 157,
 25,
 50,
 67,
 35,
 193,
 173,
 289,
 70,
 114,
 171,
 176,
 593,
 112,
 414,
 499,
 110,
 123,
 84,
 262,
 302,
 66,
 179,
 42,
 153,
 92,
 261,
 93,
 113,
 182,
 92,
 178,
 109,
 101,
 97,
 109,
 92,
 87,
 113,
 138,
 52,
 495,
 62,
 98,
 59,
 46,
 83,
 97,
 63,
 272,
 44,
 178,
 121,
 265,
 92,
 69,
 240,
 54,
 60,
 140,
 62,
 154,
 82,
 96,
 54,
 145,
 213,
 130,
 89,
 165,
 109,
 71,
 70,
 120,
 303,
 153,
 126,
 54,
 145,
 49,
 107,
 214,
 285,
 145,
 13,
 673,
 354,
 79,
 85,
 388,
 539,
 290,
 135,
 19,
 270,
 144,
 205,
 36,
 101,
 323,
 341,
 106,
 125,
 162,
 103,
 74,
 91,
 181,
 130,
 101,
 50,
 111,
 14,
 53,
 120,
 97,
 61,
 92,
 62,
 85,
 84,
 63,
 82,
 65,
 38,
 80,
 172,
 54,
 256,
 288,
 26,
 190,
 20,
 173,
 52,
 152,
 384,
 136,
 28,
 100,
 94,
 381,
 66,
 

We count the 90% of the words for each description in descriptionlist1

In [692]:
lenght_90 = []
for i in lenght:
    j = round(i * 0.9)
    lenght_90.append(j)
lenght_90

[39,
 92,
 98,
 98,
 71,
 32,
 6,
 459,
 190,
 130,
 98,
 155,
 158,
 6,
 96,
 68,
 256,
 256,
 115,
 718,
 92,
 202,
 44,
 96,
 134,
 115,
 218,
 338,
 97,
 106,
 62,
 96,
 104,
 202,
 218,
 21,
 141,
 22,
 45,
 60,
 32,
 174,
 156,
 260,
 63,
 103,
 154,
 158,
 534,
 101,
 373,
 449,
 99,
 111,
 76,
 236,
 272,
 59,
 161,
 38,
 138,
 83,
 235,
 84,
 102,
 164,
 83,
 160,
 98,
 91,
 87,
 98,
 83,
 78,
 102,
 124,
 47,
 446,
 56,
 88,
 53,
 41,
 75,
 87,
 57,
 245,
 40,
 160,
 109,
 238,
 83,
 62,
 216,
 49,
 54,
 126,
 56,
 139,
 74,
 86,
 49,
 130,
 192,
 117,
 80,
 148,
 98,
 64,
 63,
 108,
 273,
 138,
 113,
 49,
 130,
 44,
 96,
 193,
 256,
 130,
 12,
 606,
 319,
 71,
 76,
 349,
 485,
 261,
 122,
 17,
 243,
 130,
 184,
 32,
 91,
 291,
 307,
 95,
 112,
 146,
 93,
 67,
 82,
 163,
 117,
 91,
 45,
 100,
 13,
 48,
 108,
 87,
 55,
 83,
 56,
 76,
 76,
 57,
 74,
 58,
 34,
 72,
 155,
 49,
 230,
 259,
 23,
 171,
 18,
 156,
 47,
 137,
 346,
 122,
 25,
 90,
 85,
 343,
 59,
 380,
 198,
 51,
 32,

For each group of dogs we compare the descriptions of the dogs (in pairs) in order too see if they share the 90% of words in the description field 

In [693]:
from collections import Counter
number = 0
number1 = 0
number2 = 0

for j in range(0, groups):
    if rangelist_2[j+1] - rangelist_2[j] > 1:
        number += 1
        print(f"Breed: {listbreed[j]} and sex: {listsex[j]}")
        conta = 0
        for i in range(rangelist_2[j], rangelist_2[j+1]):
            descriptionlist2 = []
            descriptionlist2 = descriptionlist1.copy()
            descriptionlist2.pop(i)
            for k in range(i,rangelist_2[j+1]-1):
                
                if lenght_90[k+1] <= lenght_90[i] <= lenght[k+1] or lenght_90[i] <= lenght_90[k+1] <= lenght[i]:
                    c1 = Counter(descriptionlist1[i].split())
                    c2 = Counter(descriptionlist2[k].split())
                    c3 = dict(c1 & c2)

                    if sum(c3.values()) >= lenght_90[i] and sum(c3.values()) >= lenght_90[k+1]:    
                        conta += 1
                        print(f"{listid[i]} and {listid[k+1]} are duplicates and share {sum(c3.values())} words in the description field")

        if conta == 0:
            print(f"In the '{listbreed[j]}' and '{listsex[j]}' dogs group there isn't any couple of duplicates\n")
            number1 += 1
        if conta == 1:
            print(f"In the '{listbreed[j]}' and '{listsex[j]}' dogs group there is {conta} couple of duplicates\n")
            number2 += 1
        if conta > 1:
            print(f"In the '{listbreed[j]}' and '{listsex[j]}' dogs group there are {conta} couples of duplicates\n")
            number2 += 1

print(f"Total possible dogs groups with duplicates: {number}\nThere are '{number1}' dogs groups without duplicates\nThere are '{number2}' dogs groups with duplicates")

Breed: Affenpinscher and sex: Female
45970614 and 45871731 are duplicates and share 108 words in the description field
In the 'Affenpinscher' and 'Female' dogs group there is 1 couple of duplicates

Breed: Affenpinscher and sex: Male
In the 'Affenpinscher' and 'Male' dogs group there isn't any couple of duplicates

Breed: Afghan Hound and sex: Male
In the 'Afghan Hound' and 'Male' dogs group there isn't any couple of duplicates

Breed: Airedale Terrier and sex: Female
In the 'Airedale Terrier' and 'Female' dogs group there isn't any couple of duplicates

Breed: Airedale Terrier and sex: Male
In the 'Airedale Terrier' and 'Male' dogs group there isn't any couple of duplicates

Breed: Akbash and sex: Female
In the 'Akbash' and 'Female' dogs group there isn't any couple of duplicates

Breed: Akita and sex: Female
In the 'Akita' and 'Female' dogs group there isn't any couple of duplicates

Breed: Akita and sex: Male
45842497 and 45842477 are duplicates and share 286 words in the descriptio

45932298 and 45931729 are duplicates and share 192 words in the description field
45570899 and 45560548 are duplicates and share 68 words in the description field
45982056 and 45790697 are duplicates and share 44 words in the description field
45881350 and 44790314 are duplicates and share 236 words in the description field
45881350 and 44545963 are duplicates and share 234 words in the description field
44790314 and 44545963 are duplicates and share 236 words in the description field
45733110 and 45732409 are duplicates and share 199 words in the description field
In the 'American Bulldog' and 'Female' dogs group there are 91 couples of duplicates

Breed: American Bulldog and sex: Male
46021164 and 46021152 are duplicates and share 5 words in the description field
46020194 and 46020135 are duplicates and share 44 words in the description field
45074118 and 43853983 are duplicates and share 41 words in the description field
45782430 and 45782390 are duplicates and share 156 words in th

45931696 and 45931670 are duplicates and share 192 words in the description field
45931696 and 45931656 are duplicates and share 192 words in the description field
45931696 and 45931611 are duplicates and share 192 words in the description field
45931696 and 45931579 are duplicates and share 192 words in the description field
45931696 and 45931524 are duplicates and share 192 words in the description field
45931670 and 45931656 are duplicates and share 192 words in the description field
45931670 and 45931611 are duplicates and share 192 words in the description field
45931670 and 45931579 are duplicates and share 192 words in the description field
45931670 and 45931524 are duplicates and share 192 words in the description field
45931656 and 45931611 are duplicates and share 192 words in the description field
45931656 and 45931579 are duplicates and share 192 words in the description field
45931656 and 45931524 are duplicates and share 192 words in the description field
45931611 and 459

45880585 and 45880468 are duplicates and share 159 words in the description field
38447590 and 35234154 are duplicates and share 201 words in the description field
45798755 and 45798488 are duplicates and share 41 words in the description field
44531948 and 44531923 are duplicates and share 55 words in the description field
41812512 and 41812506 are duplicates and share 10 words in the description field
41812512 and 41717430 are duplicates and share 10 words in the description field
41812512 and 41695358 are duplicates and share 10 words in the description field
41812512 and 41689038 are duplicates and share 10 words in the description field
41812512 and 41689025 are duplicates and share 10 words in the description field
41812512 and 41686393 are duplicates and share 10 words in the description field
41812512 and 41686364 are duplicates and share 10 words in the description field
41812512 and 41686348 are duplicates and share 10 words in the description field
41812512 and 41684090 are 

45849598 and 45849562 are duplicates and share 42 words in the description field
45901429 and 45901401 are duplicates and share 142 words in the description field
45901429 and 45901338 are duplicates and share 142 words in the description field
45901429 and 45901328 are duplicates and share 142 words in the description field
45901429 and 45901264 are duplicates and share 142 words in the description field
45901401 and 45901338 are duplicates and share 142 words in the description field
45901401 and 45901328 are duplicates and share 142 words in the description field
45901401 and 45901264 are duplicates and share 142 words in the description field
45901338 and 45901328 are duplicates and share 142 words in the description field
45901338 and 45901264 are duplicates and share 142 words in the description field
45901328 and 45901264 are duplicates and share 142 words in the description field
45950569 and 45468438 are duplicates and share 67 words in the description field
46014068 and 46014

...

46038385 and 46037920 are duplicates and share 79 words in the description field
46038385 and 46037915 are duplicates and share 79 words in the description field
46038385 and 46037850 are duplicates and share 79 words in the description field
46038385 and 45882817 are duplicates and share 79 words in the description field
46038385 and 45882733 are duplicates and share 75 words in the description field
46037920 and 46037915 are duplicates and share 79 words in the description field
46037920 and 46037850 are duplicates and share 79 words in the description field
46037920 and 45882817 are duplicates and share 79 words in the description field
46037920 and 45882733 are duplicates and share 75 words in the description field
46037915 and 46037850 are duplicates and share 79 words in the description field
46037915 and 45882817 are duplicates and share 79 words in the description field
46037915 and 45882733 are duplicates and share 75 words in the description field
46037850 and 45882817 are du

45733106 and 45733072 are duplicates and share 154 words in the description field
45733106 and 45733002 are duplicates and share 154 words in the description field
45733106 and 45732861 are duplicates and share 154 words in the description field
45733106 and 45629070 are duplicates and share 154 words in the description field
45733072 and 45733002 are duplicates and share 154 words in the description field
45733072 and 45732861 are duplicates and share 154 words in the description field
45733072 and 45629070 are duplicates and share 154 words in the description field
45733002 and 45732861 are duplicates and share 154 words in the description field
45733002 and 45629070 are duplicates and share 154 words in the description field
45732861 and 45629070 are duplicates and share 154 words in the description field
46039980 and 46018270 are duplicates and share 43 words in the description field
46039980 and 45812373 are duplicates and share 43 words in the description field
46039980 and 45812

45494916 and 45481453 are duplicates and share 171 words in the description field
45532544 and 45300749 are duplicates and share 433 words in the description field
45532544 and 45300757 are duplicates and share 433 words in the description field
45532544 and 45300762 are duplicates and share 431 words in the description field
45300749 and 45300757 are duplicates and share 431 words in the description field
45300749 and 45300762 are duplicates and share 430 words in the description field
45300757 and 45300762 are duplicates and share 429 words in the description field
46029524 and 46029521 are duplicates and share 63 words in the description field
46025230 and 46025210 are duplicates and share 19 words in the description field
46038082 and 46038077 are duplicates and share 79 words in the description field
45820378 and 45820429 are duplicates and share 80 words in the description field
45820378 and 45820439 are duplicates and share 80 words in the description field
45820378 and 45820452

46019465 and 45800396 are duplicates and share 56 words in the description field
45990311 and 45985876 are duplicates and share 166 words in the description field
In the 'Terrier' and 'Male' dogs group there are 127 couples of duplicates

Breed: Thai Ridgeback and sex: Male
In the 'Thai Ridgeback' and 'Male' dogs group there isn't any couple of duplicates

Breed: Tibetan Mastiff and sex: Female
45825645 and 45825635 are duplicates and share 58 words in the description field
In the 'Tibetan Mastiff' and 'Female' dogs group there is 1 couple of duplicates

Breed: Tibetan Spaniel and sex: Male
In the 'Tibetan Spaniel' and 'Male' dogs group there isn't any couple of duplicates

Breed: Tibetan Terrier and sex: Male
In the 'Tibetan Terrier' and 'Male' dogs group there isn't any couple of duplicates

Breed: Toy Fox Terrier and sex: Female
45917778 and 45917775 are duplicates and share 153 words in the description field
In the 'Toy Fox Terrier' and 'Female' dogs group there is 1 couple of dupl

In the 'Yorkshire Terrier' and 'Female' dogs group there are 10 couples of duplicates

Breed: Yorkshire Terrier and sex: Male
43971538 and 43971630 are duplicates and share 163 words in the description field
45572590 and 45572522 are duplicates and share 347 words in the description field
45451786 and 45451610 are duplicates and share 341 words in the description field
45690903 and 45690883 are duplicates and share 344 words in the description field
46037706 and 46037440 are duplicates and share 135 words in the description field
40616088 and 40616074 are duplicates and share 162 words in the description field
40616088 and 40615973 are duplicates and share 164 words in the description field
40616074 and 40615973 are duplicates and share 162 words in the description field
45958024 and 45957955 are duplicates and share 771 words in the description field
In the 'Yorkshire Terrier' and 'Male' dogs group there are 9 couples of duplicates

Total possible dogs groups with duplicates: 358
Ther